In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from pandas.plotting import scatter_matrix
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/public_timeseries_testing_util.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/__init__.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test

* #### Get the relevant training datasets and combine them to form the full training set with Proteins and Peptides
* #### Remove the NaNs from the UPDRS test scores, using the mean from within the same group of UPDRS test scores, e.g. NaNs in updrs_4 are filled with mean of updrs_4
* #### Create new mean and median based features and get rid of Proteins Expressions (NPX) and Peptide Abundance

In [2]:
train_proteins_df = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')

train_peptides_df = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv')

pro_pep_join = pd.merge(train_proteins_df, train_peptides_df, on=['patient_id','visit_id','visit_month','UniProt'], how='inner')

del train_proteins_df, train_peptides_df
gc.collect()

train_clinical_data_df = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')

merged_data = pd.merge(pro_pep_join, train_clinical_data_df, on=['patient_id','visit_id','visit_month'], how='inner')

del pro_pep_join, train_clinical_data_df
gc.collect()

full_training_data = pd.DataFrame()

for i in [0,6,12,24]:
    temp = merged_data[merged_data['visit_month']==i]
    full_training_data = full_training_data.append(temp)
    
full_training_data = full_training_data.drop(columns=['UniProt','Peptide','upd23b_clinical_state_on_medication','patient_id'])

list_of_updrs_tests = ['updrs_1','updrs_2','updrs_3','updrs_4']

for i in range(len(list_of_updrs_tests)):
    full_training_data[list_of_updrs_tests[i]].fillna(full_training_data[list_of_updrs_tests[i]].mean().round(decimals = 0), inplace=True)

new_features = full_training_data.groupby(['visit_id','visit_month']).agg(npx_mean=('NPX', 'mean'), npx_median=('NPX', 'median')
                                                                          , peptide_mean=('PeptideAbundance', 'mean'), peptide_median=('PeptideAbundance', 'median')
                                                          ).reset_index(level=['visit_id','visit_month'])

full_training_data = pd.merge(full_training_data, new_features, on=['visit_id','visit_month'], how='inner')

#full_training_data['NPX_standardised'] = (full_training_data['NPX'] - full_training_data['NPX'].mean())/full_training_data['NPX'].std()
#full_training_data['Pep_standardised'] = (full_training_data['PeptideAbundance'] - full_training_data['PeptideAbundance'].mean())/full_training_data['PeptideAbundance'].std()

full_training_data = full_training_data.drop(columns=['NPX','PeptideAbundance'])
full_training_data = full_training_data.drop_duplicates(keep='first')
full_training_data = full_training_data[['visit_id','visit_month','npx_mean','npx_median','peptide_mean','peptide_median','updrs_1','updrs_2','updrs_3','updrs_4']]
#full_training_data = full_training_data[['visit_id','visit_month','NPX_standardised','Pep_standardised','updrs_1','updrs_2','updrs_3','updrs_4']]

full_training_data = full_training_data.reset_index()
full_training_data.head(10)

,index,visit_id,visit_month,npx_mean,npx_median,peptide_mean,peptide_median,updrs_1,updrs_2,updrs_3,updrs_4
0,0,55_0,0,2.028789e+07,1221530.0,748153.907014,93134.80,10.0,6.0,15.0,2.0
1,931,1517_0,0,1.579576e+07,897182.0,618823.150490,65253.60,11.0,6.0,25.0,5.0
2,1833,1923_0,0,1.889800e+07,1299230.0,763459.201459,92280.90,2.0,0.0,0.0,2.0
3,2710,2660_0,0,1.298973e+07,1078570.0,532250.381374,73461.05,2.0,0.0,0.0,2.0
4,3620,3636_0,0,1.367422e+07,776581.0,501743.931250,54660.20,1.0,2.0,9.0,2.0
5,4495,3863_0,0,2.279134e+07,885864.0,806928.584862,78719.90,8.0,13.0,36.0,4.0
6,5291,4161_0,0,1.647734e+07,1067620.0,699803.391029,77037.80,6.0,1.0,0.0,2.0
7,6191,4172_0,0,1.729472e+07,874171.0,620558.476752,68208.05,2.0,0.0,0.0,2.0
8,7115,5027_0,0,2.281632e+07,1134880.0,764893.568584,71828.35,1.0,0.0,0.0,2.0
9,8025,5178_0,0,1.815310e+07,998682.0,689200.799757,79225.50,9.0,1.0,3.0,2.0


#### Check for NaNs

In [3]:
full_training_data.isna().sum()

index             0
visit_id          0
visit_month       0
npx_mean          0
npx_median        0
peptide_mean      0
peptide_median    0
updrs_1           0
updrs_2           0
updrs_3           0
updrs_4           0
dtype: int64

#### We will attempt to train one model each for each of the UPDRS tests. To do that, we will also proceed to create a train-test split on the Training dataset specific to each of the UPDRS test, to test out the accuracy of the developed models
#### We will store all the models in a dictionary
#### The following cells (except the declaration of the model dict) will be repeated for all UPDRS tests

In [ ]:
"""model_dict = {}

for i in [1,2,3,4]:
    if (i == 1) | (i == 2):
        full_training_data['updrs_'+str(i)+'_category'] = pd.cut(full_training_data['updrs_'+str(i)],bins=[-np.inf, 15., np.inf],labels=[1,2])
    elif i == 3:
        full_training_data['updrs_'+str(i)+'_category'] = pd.cut(full_training_data['updrs_'+str(i)],bins=[-np.inf, 20., 40., np.inf],labels=[1,2,3])
    else:
        full_training_data['updrs_'+str(i)+'_category'] = pd.cut(full_training_data['updrs_'+str(i)],bins=[-np.inf, 7.5, np.inf],labels=[1,2])
        
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(full_training_data, full_training_data['updrs_'+str(i)+'_category']):
        strat_train_set = full_training_data.loc[train_index]
        strat_test_set = full_training_data.loc[test_index]
        
    if i == 1:
        strat_train_set = strat_train_set.drop(columns=['updrs_1_category'])
        strat_test_set = strat_test_set.drop(columns=['updrs_1_category'])
    elif i == 2:
        strat_train_set = strat_train_set.drop(columns=['updrs_1_category','updrs_2_category'])
        strat_test_set = strat_test_set.drop(columns=['updrs_1_category','updrs_2_category'])
    elif i == 3:
        strat_train_set = strat_train_set.drop(columns=['updrs_1_category','updrs_2_category','updrs_3_category'])
        strat_test_set = strat_test_set.drop(columns=['updrs_1_category','updrs_2_category','updrs_3_category'])
    else:
        strat_train_set = strat_train_set.drop(columns=['updrs_1_category','updrs_2_category','updrs_3_category','updrs_4_category'])
        strat_test_set = strat_test_set.drop(columns=['updrs_1_category','updrs_2_category','updrs_3_category','updrs_4_category'])
    
    strat_train_set_wo_labels = strat_train_set.drop(columns=['updrs_1','updrs_2','updrs_3','updrs_4','index'])
    strat_train_set_labels = strat_train_set['updrs_'+str(i)].copy()
    strat_train_set_numeric = strat_train_set_wo_labels.drop(columns=['visit_id','visit_month'])
    strat_train_set_cat = strat_train_set_wo_labels[['visit_month']]

    strat_test_set_wo_labels = strat_test_set.drop(columns=['updrs_1','updrs_2','updrs_3','updrs_4','index'])
    strat_test_set_labels = strat_test_set['updrs_'+str(i)].copy()
    
    numeric_pipeline = Pipeline([
        ('standard_scaler', StandardScaler())
    ])

    numeric_attributes = list(strat_train_set_numeric)
    categorical_attributes = list(strat_train_set_cat)

    full_pipeline = ColumnTransformer([
        ('num', numeric_pipeline, numeric_attributes),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_attributes)
    ])

    strat_train_set_prepared = full_pipeline.fit_transform(strat_train_set_wo_labels)
    print(strat_train_set_prepared.shape)
    strat_test_set_prepared = full_pipeline.transform(strat_test_set_wo_labels)
    print(strat_test_set_prepared.shape)
    
    lin_reg = LinearRegression()

    model_dict['updrs_'+str(i)] = lin_reg.fit(strat_train_set_prepared, strat_train_set_labels)
    print(model_dict)"""

In [ ]:
"""model_dict = {}

for u in list_of_updrs_tests:
        
    # Drop NAs
    temp = full_training_data.dropna(subset=[u]) 
    print(u)
    # Train data
    #X = temp[['visit_month','npx_mean','npx_median','peptide_mean','peptide_median']]
    X = temp[['visit_month','NPX_standardised','Pep_standardised']]
    print(X.shape)
    y = temp[u]
    print(y.shape)
    trained = LinearRegression().fit(X, y)
    
    # Save model
    model_dict[u] = trained"""

In [3]:
poly_model_dict = {}

for u in list_of_updrs_tests:
        
    # Drop NAs
    temp = full_training_data.dropna(subset=[u]) 
    print(u)
    # Train data
    X = temp[['visit_month','npx_mean','npx_median','peptide_mean','peptide_median']]
    #X = temp[['visit_month','NPX_standardised','Pep_standardised']]
    print(X.shape)
    y = temp[u]
    print(y.shape)
    poly = PolynomialFeatures(degree = 7)
    X_poly = poly.fit_transform(X)
    poly.fit(X_poly, y)
    trained = LinearRegression().fit(X_poly, y)
    
    # Save model
    poly_model_dict[u] = trained

updrs_1
(632, 5)
(632,)
updrs_2
(632, 5)
(632,)
updrs_3
(632, 5)
(632,)
updrs_4
(632, 5)
(632,)


In [3]:
"""decision_model_dict = {}

for u in list_of_updrs_tests:
        
    # Drop NAs
    temp = full_training_data.dropna(subset=[u]) 
    print(u)
    # Train data
    X = temp[['visit_month','npx_mean','npx_median','peptide_mean','peptide_median']]
    print(X.shape)
    y = temp[u]
    print(y.shape)
    trained = DecisionTreeRegressor().fit(X, y)
    
    # Save model
    decision_model_dict[u] = trained"""

updrs_1
(632, 5)
(632,)
updrs_2
(632, 5)
(632,)
updrs_3
(632, 5)
(632,)
updrs_4
(632, 5)
(632,)


#### The commands below are for debugging only, these were used to determine the bins corresponding to the train-test split specific to UPDRS tests

In [4]:
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))

    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)

    pos_ind = dem != 0
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]

    return 100 * np.mean(smap)

"""train_true_all = [1,2,3,4,150,0,0,0,0]
train_yhat_all = [0,0,0,0,0,0,0,0,0]
print('SMAPE: ',smape(np.array(train_true_all), np.round(np.array(train_yhat_all))))
print('SMAPE+1: ',smape(np.array(train_true_all)+1, np.round(np.array(train_yhat_all)+1)))"""

"train_true_all = [1,2,3,4,150,0,0,0,0]\ntrain_yhat_all = [0,0,0,0,0,0,0,0,0]\nprint('SMAPE: ',smape(np.array(train_true_all), np.round(np.array(train_yhat_all))))\nprint('SMAPE+1: ',smape(np.array(train_true_all)+1, np.round(np.array(train_yhat_all)+1)))"

In [ ]:
"""for u in list_of_updrs_tests:
    
    #X = full_training_data[['visit_month','npx_mean','npx_median','peptide_mean','peptide_median']]
    X = full_training_data[['visit_month','NPX_standardised','Pep_standardised']]

    y_true = full_training_data[u]
    y_pred = np.ceil(model_dict[u].predict(X))
    
    print('Linear regression SMAPE + 1 for', u, ':', smape(np.array(y_true), np.array(y_pred)))"""

In [9]:
for u in list_of_updrs_tests:
    
    X = full_training_data[['visit_month','npx_mean','npx_median','peptide_mean','peptide_median']]
    #X = full_training_data[['visit_month','NPX_standardised','Pep_standardised']]
    poly = PolynomialFeatures(degree = 7)
    X_poly = poly.fit_transform(X)
    
    y_true = full_training_data[u]
    y_pred = np.ceil(poly_model_dict[u].predict(X_poly))
    
    print('Polynomial regression SMAPE + 1 for', u, ':', smape(np.array(y_true), np.array(y_pred)))

Polynomial regression SMAPE + 1 for updrs_1 : 71.77481087470822
Polynomial regression SMAPE + 1 for updrs_2 : 90.73058467835861
Polynomial regression SMAPE + 1 for updrs_3 : 81.75176110594164
Polynomial regression SMAPE + 1 for updrs_4 : 70.88299025166116


In [5]:
"""for u in list_of_updrs_tests:
    
    X = full_training_data[['visit_month','npx_mean','npx_median','peptide_mean','peptide_median']]

    y_true = full_training_data[u]
    y_pred = np.ceil(decision_model_dict[u].predict(X))
    
    print('Decision Tree regression SMAPE + 1 for', u, ':', smape(np.array(y_true), np.array(y_pred)))
    
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print('RMSE with Decision Tree Regressor model', rmse)"""

Decision Tree regression SMAPE + 1 for updrs_1 : 0.0
RMSE with Decision Tree Regressor model 0.0
Decision Tree regression SMAPE + 1 for updrs_2 : 0.0
RMSE with Decision Tree Regressor model 0.0
Decision Tree regression SMAPE + 1 for updrs_3 : 0.0
RMSE with Decision Tree Regressor model 0.0
Decision Tree regression SMAPE + 1 for updrs_4 : 0.0
RMSE with Decision Tree Regressor model 0.0


In [ ]:
#full_training_data[['updrs_1','updrs_2','updrs_3','updrs_4']].hist(figsize=(20,15))

In [ ]:
"""full_training_data['updrs_2_category'] = pd.cut(full_training_data['updrs_2'],
                                                          bins=[-np.inf, 15., np.inf],
                                                          labels=[1,2])

full_training_data['updrs_3_category'] = pd.cut(full_training_data['updrs_3'],
                                                          bins=[-np.inf, 20., 40., np.inf],
                                                          labels=[1,2,3])

full_training_data['updrs_4_category'] = pd.cut(full_training_data['updrs_4'],
                                                          bins=[-np.inf, 7.5, np.inf],
                                                          labels=[1,2])

full_training_data['updrs_4_category'].hist()

full_training_data['updrs_4_category'].unique()
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(full_training_data, full_training_data['updrs_4_category']):
    strat_train_set = full_training_data.loc[train_index]
    strat_test_set = full_training_data.loc[test_index]
    
print(strat_test_set['updrs_4_category'].value_counts()/len(strat_test_set))
print(strat_train_set['updrs_4_category'].value_counts()/len(strat_train_set))

"""

### Testing the RMSE
#### RMSE <= 0.75 : Very good accuracy | 0.75 < RMSE <= 1.0 : Good accuracy

#### First on train data (train split from the full training dataset)
#### 2nd on test data (test split from the full training dataset)

In [ ]:
"""train_df_predictions = lin_reg_model_updrs_4.predict(strat_train_set_prepared)
mse = mean_squared_error(strat_train_set_labels, train_df_predictions)
rmse = np.sqrt(mse)
print(rmse)

test_df_predictions = lin_reg_model_updrs_4.predict(strat_test_set_prepared)
mse = mean_squared_error(strat_test_set_labels, test_df_predictions)
rmse = np.sqrt(mse)
print(rmse)

model_dict['updrs_4'] = lin_reg_model_updrs_4
model_dict"""

#### The below cell is only for debugging the function get_predictions()

In [6]:
def get_full_test_data(df_test, df_proteins=pd.DataFrame(), df_peptides=pd.DataFrame()):
    
    if  (df_proteins.shape[0] == 0) & (df_peptides.shape[0] == 0):
        
        print('only the test dataframe has data, proteins and peptides info absent')
        full_test_data = df_test[['visit_id','visit_month']]
        full_test_data['npx_mean'] = 0
        full_test_data['npx_median'] = 0
        full_test_data['peptide_mean'] = 0
        full_test_data['peptide_median'] = 0
        
        full_test_data = full_test_data.drop_duplicates(keep='first')
        full_test_data = full_test_data[['visit_id','visit_month','npx_mean','npx_median','peptide_mean','peptide_median']]

        full_test_data = full_test_data.reset_index()
        full_test_data = full_test_data.drop(columns=['index'])
        return full_test_data
    
    elif (df_proteins.shape[0] == 0) & (df_peptides.shape[0] != 0):
        
        print('no proteins only peptides')
        full_test_data = pd.DataFrame()
        
        df_peptides = pd.merge(df_peptides, df_test, on=['patient_id','visit_id','visit_month'], suffixes=['_left','_right'], how='outer')
        df_peptides['PeptideAbundance'] = np.where(df_peptides['PeptideAbundance'].isna(), 0, df_peptides['PeptideAbundance'])

        for i in [0,6,12,24]:
            temp = df_peptides[df_peptides['visit_month']==i]
            full_test_data = full_test_data.append(temp)
        
        del df_proteins, df_peptides
            
        new_features = full_test_data.groupby(['visit_id','visit_month']).agg(peptide_mean=('PeptideAbundance', 'mean'), peptide_median=('PeptideAbundance', 'median')
                                                              ).reset_index(level=['visit_id','visit_month'])

        full_test_data = pd.merge(full_test_data, new_features, on=['visit_id','visit_month'], how='inner')
        full_test_data['npx_mean'] = 0
        full_test_data['npx_median'] = 0
        full_test_data = full_test_data.drop_duplicates(keep='first')
        full_test_data = full_test_data[['visit_id','visit_month','npx_mean','npx_median','peptide_mean','peptide_median']]

        full_test_data = full_test_data.reset_index()
        full_test_data = full_test_data.drop(columns=['index'])
        
        return full_test_data
            
    elif (df_peptides.shape[0] == 0) & (df_proteins.shape[0] != 0):
        
        print('no peptides only proteins')
        full_test_data = pd.DataFrame()
        
        df_proteins = pd.merge(df_proteins, df_test, on=['patient_id','visit_id','visit_month'], suffixes=['_left','_right'], how='outer')
        df_proteins['NPX'] = np.where(df_proteins['NPX'].isna(), 0, df_proteins['NPX'])

        for i in [0,6,12,24]:
            temp = df_proteins[df_proteins['visit_month']==i]
            full_test_data = full_test_data.append(temp)
            
        new_features = full_test_data.groupby(['visit_id','visit_month']).agg(npx_mean=('NPX', 'mean'), npx_median=('NPX', 'median')
                                                              ).reset_index(level=['visit_id','visit_month'])

        full_test_data = pd.merge(full_test_data, new_features, on=['visit_id','visit_month'], how='inner')
        full_test_data['peptide_mean'] = 0
        full_test_data['peptide_median'] = 0
        full_test_data = full_test_data.drop_duplicates(keep='first')
        full_test_data = full_test_data[['visit_id','visit_month','npx_mean','npx_median','peptide_mean','peptide_median']]

        full_test_data = full_test_data.reset_index()
        full_test_data = full_test_data.drop(columns=['index'])
        
        return full_test_data
        
    else:
        
        print('both proteins and peptides are present')
        
        pro_pep_join = pd.merge(df_proteins, df_peptides, on=['patient_id','visit_id','visit_month','UniProt'], suffixes=['_left','_right'], how='outer')
        del df_proteins, df_peptides
        
        pro_pep_join_test = pd.merge(pro_pep_join, df_test, on=['patient_id','visit_id','visit_month'], suffixes=['_left','_right'], how='outer')
        
        pro_pep_join_test['NPX'] = np.where(pro_pep_join_test['NPX'].isna(), 0, pro_pep_join_test['NPX'])
        pro_pep_join_test['PeptideAbundance'] = np.where(pro_pep_join_test['PeptideAbundance'].isna(), 0, pro_pep_join_test['PeptideAbundance'])
        
        del pro_pep_join
        gc.collect()

        full_test_data = pd.DataFrame()

        for i in [0,6,12,24]:
            temp = pro_pep_join_test[pro_pep_join_test['visit_month']==i]
            full_test_data = full_test_data.append(temp)

        new_features = full_test_data.groupby(['visit_id','visit_month']).agg(npx_mean=('NPX', 'mean'), npx_median=('NPX', 'median')
                                                                                  , peptide_mean=('PeptideAbundance', 'mean')
                                                                              , peptide_median=('PeptideAbundance', 'median')
                                                                  ).reset_index(level=['visit_id','visit_month'])

        full_test_data = pd.merge(full_test_data, new_features, on=['visit_id','visit_month'], how='inner')
        full_test_data = full_test_data.drop_duplicates(keep='first')
        full_test_data = full_test_data[['visit_id','visit_month','npx_mean','npx_median','peptide_mean','peptide_median']]

        full_test_data = full_test_data.reset_index()
        full_test_data = full_test_data.drop(columns=['index'])
        
        return full_test_data

In [14]:
#def get_predictions(model_dict, sklearn_pipeline, df_test, df_proteins=pd.DataFrame(), df_peptides=pd.DataFrame()):
def get_predictions(model_dict, df_test, df_proteins=pd.DataFrame(), df_peptides=pd.DataFrame()):
    
    list_of_updrs_tests = ['updrs_1','updrs_2','updrs_3','updrs_4']
    result = pd.DataFrame()
    
    full_test_data = get_full_test_data(df_test=df_test, df_proteins=df_proteins, df_peptides=df_peptides)
    
    if full_test_data.shape[0] != 0:
        """df_prepared = sklearn_pipeline.transform(full_test_data)
        print(df_prepared.shape)"""

        for u in list_of_updrs_tests:
            X = full_test_data[['visit_month','npx_mean','npx_median','peptide_mean','peptide_median']]
            ## for poly model
            poly = PolynomialFeatures(degree = 7)
            X_poly = poly.fit_transform(X)
            full_test_data['result_' + str(u)] = np.ceil(model_dict[u].predict(X_poly))
            #full_test_data['result_' + str(u)] = pd.DataFrame(model_dict[u].predict(df_prepared),columns=['rating']).round(decimals=0)
            ## for all other models
            #full_test_data['result_' + str(u)] = np.ceil(model_dict[u].predict(X))

        for m in [0, 6, 12, 24]:
            for u in [1, 2, 3, 4]:
                temp = full_test_data[['visit_id', 'visit_month', 'result_updrs_' + str(u)]]
                temp['prediction_id'] = temp['visit_id'] + '_updrs_' + str(u) + '_plus_' + str(m) + '_months'
                
                if u == 4:
                    temp["rating"] = 0
                else:
                    temp["rating"] = temp["result_updrs_" + str(u)]
                
                #temp = temp[['visit_id', 'visit_month', 'prediction_id', 'rating']]
                temp = temp[['prediction_id', 'rating']]

                result = result.append(temp)

        #result.sort_values(by=['visit_id', 'visit_month'], inplace=True)
        #result = result[['prediction_id', 'rating']]
        #result['prediction_id'] = result['prediction_id'].astype('string') 
        #result['rating'] = result['rating'].astype('int') 
        result = result.drop_duplicates(subset=['prediction_id', 'rating'])
        result = result.reset_index()
        result.drop(columns=['index'], inplace=True)
    
    return result

In [15]:
"""df_test = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test.csv')

test_proteins = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv')
#result = get_predictions(model_dict=model_dict, sklearn_pipeline=full_pipeline, df_test=df_test, df_proteins=test_proteins, df_peptides=pd.DataFrame())

test_peptides = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv')
#result = get_predictions(model_dict=model_dict, sklearn_pipeline=full_pipeline, df_test=df_test, df_proteins=pd.DataFrame(), df_peptides=test_peptides)

#result = get_predictions(model_dict=model_dict, sklearn_pipeline=full_pipeline, df_test=df_test, df_proteins=test_proteins, df_peptides=test_peptides)
result = get_predictions(model_dict=poly_model_dict, df_test=df_test)

#result = get_predictions(model_dict=model_dict, sklearn_pipeline=full_pipeline)

result"""

only the test dataframe has data, proteins and peptides info absent


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,prediction_id,rating
0,3342_0_updrs_1_plus_0_months,8.0
1,50423_0_updrs_1_plus_0_months,8.0
2,3342_6_updrs_1_plus_0_months,8.0
3,50423_6_updrs_1_plus_0_months,8.0
4,3342_0_updrs_2_plus_0_months,9.0
...,...,...
59,50423_6_updrs_3_plus_24_months,23.0
60,3342_0_updrs_4_plus_24_months,0.0
61,50423_0_updrs_4_plus_24_months,0.0
62,3342_6_updrs_4_plus_24_months,0.0


In [8]:
import amp_pd_peptide

env = amp_pd_peptide.make_env()   # initialize the environment for one run only

#amp_pd_peptide.make_env.__called__ = False
#type(env)._state = type(type(env)._state).__dict__['INIT']

iter_test = env.iter_test()

for (test, test_peptides, test_proteins, submission) in iter_test:
    result = get_predictions(model_dict=poly_model_dict, df_test=test)
    #result = get_predictions(model_dict=decision_model_dict, df_test=test, df_peptides=test_peptides)
    #result = get_predictions(model_dict=decision_model_dict, df_test=test, df_proteins=test_proteins, df_peptides=test_peptides)
    env.predict(result)
    

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
only the test dataframe has data, proteins and peptides info absent


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


ValueError: X has 5 features, but LinearRegression is expecting 792 features as input.

In [15]:
"""submission = pd.read_csv('/kaggle/working/submission.csv')
submission.drop_duplicates()"""

,prediction_id,rating
0,50423_0_updrs_1_plus_0_months,1.0
1,3342_0_updrs_1_plus_0_months,1.0
2,50423_0_updrs_2_plus_0_months,0.0
3,3342_0_updrs_2_plus_0_months,13.0
4,50423_0_updrs_3_plus_0_months,25.0
...,...,...
60,3342_6_updrs_3_plus_24_months,35.0
61,50423_6_updrs_3_plus_24_months,68.0
62,3342_6_updrs_4_plus_24_months,2.0
63,50423_6_updrs_4_plus_24_months,0.0


In [ ]:
"""import amp_pd_peptide
env = amp_pd_peptide.make_env()

amp_pd_peptide.make_env.__called__ = False
type(env)._state = type(type(env)._state).__dict__['INIT'] # initialize the environment for multiple runs
iter_test = env.iter_test()

for (test, test_peptides, test_proteins, submission) in iter_test:
    result = get_predictions(model_dict=decision_model_dict, df_test=test, df_proteins=test_proteins, df_peptides=test_peptides)
    env.predict(result)"""